In [1]:
import tensorflow as tf
from droughts_modelling.window_gen import WindowGenerator
from droughts_modelling.data import DataFunctions

In [2]:
data = DataFunctions(local=True).light_weekly_aggregate_train()

/Users/hughlupson/.pyenv/versions/droughts/lib/python3.8/site-packages/droughts_modelling/data.py:93: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_num'] = pd.to_datetime(df['date']).dt.week


In [3]:
window = WindowGenerator(data,input_width=6,label_width=6,shift=1,label_columns=["score_max"]).make_dataset()

AttributeError: module 'tensorflow' has no attribute 'keras'

In [4]:
window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [1 2 3 4 5 6]
Label column name(s): ['score_max']

In [ ]:
from droughts_modelling.data import DataFunctions
import tensorflow as tf
import numpy as np


class WindowGenerator():
    def __init__(self,data,input_width,label_width,shift,label_columns=None):

        self.data = data
        
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
            self.column_indices = {name: i for i, name in enumerate(self.data.columns)}

    
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    
    def split_window(self, list_of_consecutive_inputs_w_labels):
        inputs = list_of_consecutive_inputs_w_labels[:, self.input_slice, :]
        labels = list_of_consecutive_inputs_w_labels[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)

        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def make_dataset(self):
        data = np.array(self.data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(data=data,targets=None,
                                                              sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=True,
          batch_size=32,)

        ds = ds.map(self.split_window)

        return ds